## CodeGrade Assignment — Tutorial and Template
_Maxim van den Berg (reach me for any questions via maximvdberg@gmail.com)_

This is tutorial and template for creating notebook assignments graded through CodeGrade. You can write the assignments, solutions and autotests all in this document. Then by running 
```bash
$ python prepare.py assignment [solution notebook].ipynb [student notebook].ipynb
``` 
you remove the solutions and autotests, and you can upload the resulting notebook for your students to Canvas.

You need to have the `nbconvert` python package installed for the script to work. 

## Table of Contents
1. [Using this document](#using-this-document)
2. [Creating autotests](#creating-autotests)
3. [Checking tests locally](#checking-tests-locally)
4. [Setting up CodeGrade](#codegrade)
5. [Student feedback](#student-feedback)
6. [Optional: Making the solutions available to students](#solutions)
7. [Optional: Link tests to specific CodeGrade rubrics](#specific-rubrics)

## 1 Using this document <a class="anchor" id="using-this-document"></a>

In these cells you write your assignment information. $\LaTeX$ is supported. In the cell below student can write their solution (although they can put in anywhere in the notebook). Make sure students use the correct variable and function names, for instance by writing them in the cell already. Do not use the same name twice in the notebook, as it breaks the autotests.

Write the solution between `#!# BEGIN SOLUTION` and `#!# END SOLUTION`. This will be automatically removed and replaced with `#. Your solution here ...` in the student notebook.

In [3]:

#!# BEGIN SOLUTION
answer = 10
#!# END SOLUTION
# answer = ...

def add(a, b):
    #!# BEGIN SOLUTION
    return a + b
    #!# END SOLUTION


You can also simply tell to student to "implement the `add(a, b)` function" and write the cell like this.

In [4]:
#!# BEGIN SOLUTION
def add(a, b):
    return a + b
#!# END SOLUTION

You can also create solution blocks in markdown cells, for instance when the students have to give a LaTeX or written answer:

#!# BEGIN SOLUTION

The answer is
$$ 
    p_X(k) = \binom{n}{k} p^k (1-p)^{n-k}.
$$

#!# END SOLUTION

## 2 Creating autotests <a class="anchor" id="creating-autotests"></a>

Write the tests between `#!# BEGIN VISIBLE TESTS` and `#!# END VISIBLE TESTS` or `#!# BEGIN HIDDEN TESTS` and `#!# END HIDDEN TESTS`. Both will be automatically removed in the student notebook. The first category will be visible for students on CodeGrade after handing in. The second will only become visible after the deadline has passed.

The tests use [pytest](https://docs.pytest.org/en/7.4.x/). Every test is a function that starts with `test_` and takes no arguments. PyTests gives full points if all `assert`-statements in the function evaluate to `True`. 

The students solutions will be available in the `student` module, so for testing the student code all variables/functions should start with `student.`. If multiple functions depend on each other, it is recommended to use `student.` only for the function being tested, to avoid continuous errors. The tests can access all variables and functions in the solution notebook directly.

The `weight`, `name` and `description` decorators link up with CodeGrade when we run the tests there. The `name` and `description` are what the students will see in CodeGrade, and should tell te students to which assignment the test belongs. Recommended is to set `name` to the assignment name (for example `"Question 1, part (a)"`). The `description` decorator is optional, but recommended to improve clarity (for example `"Test the add function"`). 

See the code below for examples and some additional testing tips.

In [26]:
#!# BEGIN VISIBLE TESTS
@weight(1)
@name("A name for the test")
@description("A description of the test")
def test_add_single():
    assert student.add(10,12) == add(10,12), "Add was incorrect" 
                                             # ^ This string will be reported to students in CodeGrade when assert fails.
#!# END VISIBLE TESTS

weight: 	 1
name: 		 A name for the test
description: 	 A description of the test
Test OK



In [27]:
#!# BEGIN VISIBLE TESTS
# You can check for strings in the student's implemention.
import re, inspect

@weight(0) # <- You can set weight to zero, just for student feedback
@name("Add function — no build-in")
@description("Check the add function doesn't use a build-in function")
def test_add_no_buildin_function():
    source = re.sub(r"#.*", "", inspect.getsource(student.add))
    assert not "np.add" in source, "Use of numpy's build-in add function is not allowed"
#!# END VISIBLE TESTS


#!# BEGIN HIDDEN TESTS
# You can add helper functions as well.
def helper_function(i):
    return i + 1

@weight(3)
@name("Add function — test multiple values")
@description("Test the add function on more inputs")
def test_add_multiple():
    for i in range(10):
        assert student.add(1, i) == helper_function(i), f"Add was incorrect on input (1, {i})"


# NumPy has some very good assert functions.
# On failure, numpy will report a nice and detailed message. 
# You can also set tolerance values, see the documentation.
import numpy as np

@weight(2.5)
@name("Add function — test multiple values")
@description("Test the add function on more inputs")
def test_add_array():
    np.testing.assert_allclose([student.add(1, i) for i in range(10)],
                               [helper_function(i) for i in range(10)])
        
#!# END HIDDEN TESTS

weight: 	 0
name: 		 Add function — no build-in
description: 	 Check the add function doesn't use a build-in function
Test OK

weight: 	 3
name: 		 Add function — test multiple values
description: 	 Test the add function on more inputs
Test OK

weight: 	 2.5
name: 		 Add function — test multiple values
description: 	 Test the add function on more inputs
Test OK



## 3 Checking tests locally <a class="anchor" id="checking-tests-locally"></a>

The `weight`,`name` and `description` decorators are not defined until running the tests in CodeGrade, which probably gave you an error when running the cell above. We can define them when developing the notebook, and also use them to automatically run the tests locally to check for correctness. You can paste the following code at the top of your notebook to do this. A `#!# BEGIN/END REMOVE_FOR TESTS` block will be removed when running the tests in CodeGrade and creating the student notebook.

In [6]:
#!# BEGIN REMOVE_FOR TESTS
class Student():
    def __init__(self):
        # Load all current variables into the student class.
        for key, value in globals().items():
            exec(f"self.{key} = {key}")
student = None

def weight(w):
    # Reload the variables into the student object.
    global student
    student = Student()
    def decorator(func):
        print("weight: \t", w)
        # By putting the function call one layer higher
        # here the test will automatically run.
        func()
        print("Test OK\n")
        def inner():
            pass
        return inner
    return decorator

def name(name):
    def decorator(func):
        def inner():
            print("name: \t\t", name)
            func()
        return inner 
    return decorator

def description(d):
    def decorator(func):
        def inner():
            print("description: \t", d)
            func()
        return inner
    return decorator

#!# END REMOVE_FOR TESTS

## 4 Setting up CodeGrade <a class="anchor" id="codegrade"></a>

### Creating the assignment

First create an assignment in Canvas, go to _Submission type_, select _External tool_ and select _CodeGra.de_ from the list. You can also set availability and deadlines here. They will be synced with CodeGrade.

![Create assignment](tutorial-images/create-assignment.png)

Now saving and opening the assignment shows the CodeGrade interface. If you want, press _New Tab_ on the bottom left to open the window in a new browser tab. Navigate to _Courses_ and then to your course. Your assignment should be listed here. Clicking on the assignment opens the submission window, which is used when grading. 

![CodeGrade window](tutorial-images/codegrade-window.png)

### Rubric

Pressing the _cogwheel_ opens the assignment settings, which is where we will configure the rubric and autotests now. 
Navigate to the _Rubric_ tab and create a new rubric. For the autotests we need to use a _Continuous category_.

![Autotest rubric](tutorial-images/rubric-continuous.png)

You can also add a manually graded category alongside the autotests, which will often be _Discrete_.

![Manually graded rubric](tutorial-images/rubric-discrete.png)

### Autotests — Setup

Now we want to setup the autotests. Navigate to the _AutoTest_ tab and select _AutoTest V2_. If you have done the following steps before, you can select _Copy_ here, but for now select _Create_.

![AutoTest Screen](tutorial-images/autotest-screen.png)

Navigate to _Setup_. The Setup steps are run once for all students, so here we will install required packages and upload required files for autograding. 
Drag the _Upload files_ step to the left and upload your notebook (including the solutions and autotests) as well as `prepare.py`.

![Setup screen](tutorial-images/setup-screen.png)

Now drag the _Script_ step to the left and copy the following line to it: 

```bash
python3 -m pip install notebook nbconvert nbval
```

Here you can install other Python packages your notebook needs as well (for instance, write `python3 -m pip install notebook nbconvert sympy numpy matplotlib` on the first line instead). 

Then drag another _Script_ step below it and copy the following line to it:
```bash
python3 $UPLOADED_FILES/prepare.py solutions
```

You can also name the steps if you want. Your setup steps should now look something like this:

![Setup scripts](tutorial-images/setup.png)

### Autotests — Tests

Next, navigate to _Tests_, drag a _Script_ step to the left and paste the following into it:
```bash
python3 $UPLOADED_FILES/prepare.py hand-in
```

Now drag a _Connect Rubric_ step to the left, and link it to the _AutoTest_ rubric category. 
Then drag a _Pytest_ step into the _Connect Rubric_ step and paste the following into it:

```python
import prepare
exec(prepare.tests("VISIBLE"))
```

This will extract all visible tests from the notebook. We can do the same for the hidden tests. For this we first drag a _Hidden_ step from the right to below the previous step, and place the same Pytest script inside it as before, but with `"VISIBLE"` changed to `"HIDDEN"`. Set the _Hidden_ step to _result | before | deadline_.

CodeGrade will assign equal weight to every Pytest step (so the visible and hidden tests will have equal weight). In many cases this is not what we want, so we will add _Weight_ steps around the two Pytest steps, and set the weight to equal the total amount of weight set in your tests. Unfortunately, you have to set this weight manually every time you change the point total.

Your steps should then look something like this: 

![Tests scripts everything](tutorial-images/tests-all.png)


### Autotests — Building a snapshot

We're almost done! Press _Build Snapshot_ on the bottom right and upload the solutions notebook as the test submission (you can reupload this under the _General_ tab). Now CodeGrade will run the setup and test steps. This might take a while.
Any error will be displayed to you. If everything went correctly you will see a list of your autotests appear. This is what students will see as well. Pressing _Publish to Students_ will start the tests for your students! 

![Autotest results](tutorial-images/test-results.png)


## 5 Student feedback <a class="anchor" id="student-feedback"></a>

Immediately after handing in, student's open up the "Convert student notebook" step in the AutoTests tab in their submission to check whether their notebook runs properly. The `F`'s here mean the corresponding cell raised an exception. 

![AutoTest Screen](tutorial-images/student-feedback-convert.png)

Failed tests will look something like this. This example uses the `np.testing.assert_allclose` function.

![AutoTest Screen](tutorial-images/student-feedback-numpy.png)

In fact, failed tests will report all `stdout` output, so any `print` statements in your tests will be visible to students when the test failed. You can use this to give more feedback. 

## 6 Optional: Making the solutions available to students <a class="anchor" id="solutions"></a>

You can remove only the tests from the notebook by running

```bash
    $ python prepare.py remove-tests [solutions notebook].ipynb [solutions but no tests notebook].ipynb
```
with `[solutions notebook.ipynb]` the notebook with the solutions and tests. You can then share the resulting file with your students on Canvas. 

## 7 Optional: Link tests to specific CodeGrade rubrics <a class="anchor" id="specific-rubrics"></a>

If you would like, you can link tests to specific rubric categories in CodeGrade, to provide more structure for your students. This is not too hard but requires some more effort to manually set things up in your notebook and CodeGrade. Because of this it is also not possible anymore to simply directly copy the CodeGrade AutoTests from a previous exercise.

The basic idea is that you can give test blocks other categories than just `HIDDEN` or `VISIBLE`. In fact, you can write any category name you like. For instance, `QUESTION1_VISIBLE`.

In [28]:
#!# BEGIN QUESTION1_VISIBLE TESTS
@weight(1)
@name("Add function 1")
@description("Testing if the add function can add 10 and 12 correctly.")
def test_add_single():
    assert student.add(10,12) == add(10,12), "Add is incorrect"
#!# END QUESTION1_VISIBLE TESTS

#!# BEGIN QUESTION1_VISIBLE TESTS
@weight(1)
@name("Add function 2")
@description("Testing if the add function can add 23 and 1 correctly.")
def test_add_single():
    assert student.add(23,1) == add(23,1), "Add is incorrect"
#!# END QUESTION1_VISIBLE TESTS


weight: 	 1
name: 		 Add function 1
description: 	 Testing if the add function can add 10 and 12 correctly.
Test OK

weight: 	 1
name: 		 Add function 2
description: 	 Testing if the add function can add 23 and 1 correctly.
Test OK



Then in the rubric add a rubric items however you would like, for instance as below. Remember to make autotested items a continuous category.

![Mutiple rubric items](tutorial-images/multiple-rubric.png)

Then in the autotest screen link up the autotests one-by-one, replacing the `"VISIBLE"` string in the script from before with whatever category names you chose (this is case sensitive!). If your question only has visible or hidden tests, you can leave out the _Weight_ steps. 
Remember to use the duplication feature (available by clicking the three dots of the _Connect Rubric_ step) to make your live easier! 

![Mutiple test items](tutorial-images/multiple-tests.png)
